In [1]:
import numpy as np
import h5py
import cleo_tools as cleo
import time 

/home/fred/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
def mass(e,px,py,pz):
    m_i = (e**2)-(px**2)+(py**2)+(pz**2)
    
    if m_i >= 0:
        m = np.sqrt(m_i)
        
    else:
        m_i *= -1
        m = np.sqrt(m_i)
        
    return m

In [4]:
#hdf5 to dictionary wrapper

t_i = time.time()

with h5py.File('cleo_test.hdf5','r') as hf:
    
    npions = hf.get('npions').value
    pions = hf.get('particles/pions').value

    nkaons = hf.get('nkaons').value
    kaons = hf.get('particles/kaons').value
    
    nmuons = hf.get('nmuons').value
    muons = hf.get('particles/muons').value
    
    nelectrons = hf.get('nelectrons').value
    electrons = hf.get('particles/electrons').value
    
    nphotons = hf.get('nphotons').value
    photons = hf.get('particles/photons').value
    
    h5_events = []
    pion = []
    kaon = []
    muon = []
    electron = []
    photon = []
    
    nlist = [npions,nkaons,nmuons,nelectrons,nphotons]
    plist = [pions,kaons,muons,electrons,photons]
    elist = [pion,kaon,muon,electron,photon]
    
    for i in range(0,len(nlist)):
        
        n_particles = nlist[i]
        particles = plist[i]
        outlist = elist[i]
        
        n_start = 0
        
        for j in n_particles:
            
            n_end = n_start + j

            event_particles = particles[n_start:n_end]
            
            for k in event_particles:
                
                ep = []
                ep.append(k)
                
            outlist.append(ep)
            
            n_start = n_end

    for l in range(0,len(pion)):
        
        ev = []
        
        ev.append(pion[l])
        ev.append(kaon[l])
        ev.append(muon[l])
        ev.append(electron[l])
        ev.append(photon[l])

        h5_events.append(ev)

print 'took %f seconds' % (time.time()-t_i)

took 1.772077 seconds


In [6]:
#convert above output to dictionaries

#stores particle information as a list of dictionaries. particle dictionaries point to subdictionaries which contain
#keys for each component of the 4-vector.
# eg: assuming i is a variable iterating through the list of dictionaries, calling i['pions'] would return a dictionary
# containing keys for "e,px,py,pz,...,etc."
t_i = time.time()

d_events = []

for i in h5_events:
    
    pions,kaons,muons,electrons,photons = i
    
    pis = []
    kas = []
    mus = []
    els = []
    phs = []
    
    for pion in pions:
        e,px,py,pz=pion[0:4]
        pidi = {"e":e,"px":px,"py":py,"pz":pz}
        pis.append(pidi)
    
    for kaon in kaons:
        e,px,py,pz=kaon[0:4]
        kadi = {"e":e,"px":px,"py":py,"pz":pz}
        kas.append(kadi)
        
    for muon in muons:
        e,px,py,pz=muon[0:4]
        mudi = {"e":e,"px":px,"py":py,"pz":pz}
        mus.append(mudi)
        
    for electron in electrons:
        e,px,py,pz=electron[0:4]
        eldi = {"e":e,"px":px,"py":py,"pz":pz}
        els.append(eldi)
        
    for photon in photons:
        e,px,py,pz=photon[0:4]
        phdi = {"e":e,"px":px,"py":py,"pz":pz}
        phs.append(phdi)

    ev = {"pions":pis,"kaons":kas,"muons":mus,"electrons":els,"photons":phs}
    
    d_events.append(ev)
    
print 'took %f seconds' % (time.time()-t_i) 

took 1.725383 seconds


In [8]:
#calculate masses using the above data storage method

t_i = time.time()

masses = []
    
for event in d_events:
    
    es = 0
    pxs = 0
    pys = 0
    pzs = 0
    
    for pion in event['pions']:
        
        es+=pion['e']
        pxs+=pion['px']
        pys+=pion['py']
        pzs+=pion['pz']
        
    for kaon in event['kaons']:
        
        es+=kaon['e']
        pxs+=kaon['px']
        pys+=kaon['py']
        pzs+=kaon['pz']
    
    for muon in event['muons']:
        
        es+=muon['e']
        pxs+=muon['px']
        pys+=muon['py']
        pzs+=muon['pz']
        
    for electron in event['electrons']:
        
        es+=electron['e']
        pxs+=electron['px']
        pys+=electron['py']
        pzs+=electron['pz']
    
    for photon in event['photons']:

        es+=photon['e']
        pxs+=photon['px']
        pys+=photon['py']
        pzs+=photon['pz']
    
    masses.append(mass(es,pxs,pys,pzs))

print 'took %f seconds' % (time.time()-t_i)

took 0.807806 seconds


In [ ]:
# old multidimensional array method of storing data. didn't work because of various issues including file size and 
# read times

'''
#data = np.zeros((100001,5,206,13))

t_i = time.time()

for i in xrange(0,len(collisions)):
    
    collision = collisions[i]
    
    pions,kaons,muons,electrons,photons=collision
    
    for j in xrange(0,len(pions)):
        
        pion = pions[j]
        
        for j_1 in xrange(0,len(pion)):
            
            part = pion[j_1]
            
            data[i][0][j][j_1] = part
            
    for k in xrange(0,len(kaons)):
        
        kaon = kaons[k]
        
        for k_1 in xrange(0,len(kaon)):
            
            part = kaon[k_1]
            
            data[i][1][k][k_1] = part
            
    for l in xrange(0,len(muons)):
        
        muon = muons[l]
        
        for l_1 in xrange(0,len(muon)):
            
            part = muon[l_1]
            
            data[i][2][l][l_1] = part
            
    for m in xrange(0,len(electrons)):
        
        electron = electrons[m]
        
        for m_1 in xrange(0,len(electron)):
            
            part = electron[m_1]
            
            data[i][3][m][m_1] = part

    for n in xrange(0,len(photons)):
        
        photon = photons[n]
        
        for n_1 in xrange(0,len(photon)):
            
            part = photon[n_1]
            
            data[i][4][n][n_1] = part
    
print "took %f seconds" % (time.time()-t_i)'''